# 종합실습2_1 따릉이

* 지금까지 배운 것을 총 복습 합니다.
* Data : 서울 공유 자전거
* 문제 : 2시간 후의 수요를 예측하고자 한다.

## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam

* 함수 만들기

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (2) 데이터로딩

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/SeoulBikeData2.csv'
data = pd.read_csv(path)
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S')
data.drop(['Visibility','Solar'], axis = 1, inplace = True)
data.head()

**변수설명**  

* DateTime : year-month-day hh:mi:ss
* Count : 시간대별 수요량
* Temperature : 온도(섭씨)
* Humidity : 습도(%)
* WindSpeed : 풍속(m/s)
* Visibility : 가시거리(단위: 10m)
* Solar - 태양복사열, MJ/m2
* Rainfall - mm
* Snowfall - cm
* Seasons - Winter, Spring, Summer, Autumn
* Holiday - Holiday / No holiday
* FuncDay - Yes / No


In [ ]:
# 데이터 기간은 다음과 같습니다.
data.DateTime.min(), data.DateTime.max()

In [ ]:
# 14일 동안의 수요량을 살펴 봅시다.
temp = data[:24*14]
plt.figure(figsize = (20,8))
plt.plot('DateTime', 'Count', data = temp)
plt.grid()
plt.show()

## 2.데이터 준비

### (1) y 만들기 
* 2시간 이후의 수요량을 예측해야 합니다.

In [ ]:
data['y'] = data['Count'].shift(-2)
data.head()

In [ ]:
# 2칸을 앞당겼기 때문에 하위 2행의 y값에 NaN이 표시되어 있습니다.
data.tail()

In [ ]:
# 하위 2행은 삭제합니다.
# 하위 2행 제외하고 다시 붓기
data = data.iloc[:-2]

### (2) 데이터 분할1 : x, y 나누기

In [ ]:
target = 'y'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) feature engineering
* 여기서는 다루지 않습니다.

In [ ]:
# 날짜 데이터 제거
x.drop('DateTime', axis = 1, inplace = True)
x.head()

### (4) NaN 조치

### (5) 가변수화

In [ ]:
cat_cols = ['Seasons','Holiday','FuncDay']
x = pd.get_dummies(x, columns = cat_cols, drop_first = True)

### (6) 데이터분할2 : train : val

* 시계열 데이터이므로 시간의 흐름에 맞게 분할합시다.
* 뒤에서 30일 : validaiton
* 나머지 : train
* 30일 : 시간단위 데이터이므로 24 * 30


In [ ]:
i = 30 * 24
x_train, x_val = x[:-i], x[-i:]
y_train, y_val = y[:-i], y[-i:]

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(y_val)
plt.show()

### (7) Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

* y 값이 클 경우 scaling 하면 학습에 도움이 됩니다.

In [ ]:
# min max scaling을 수동으로 수행합니다.
y_train_min = y_train.min()
y_train_max = y_train.max()

y_train_s = (y_train - y_train_min) / (y_train_max - y_train_min)

## 3.모델링
* 딥러닝 모델을 최소 2개 이상을 생성하고 성능을 비교해 봅시다.


* 모델1

* 모델2